## Used Car Price Prediction

### 1) Problem Statement
        - This Dataset comprises used cars sold on cardekho.com in India as well as important features of these cars.
        - If user can predict the price of the car based on the input features.
        - Prediction results can be used to give a new seller the price suggestion based on market condition.

### 2) Data Collection
    
    - The Dataset is collected from scrapping from cardekho website
    - The data consist of 13 columns and 15411 rows

In [1]:
## Importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
dataset = pd.read_csv('cardekho_imputated.csv', index_col=0)

In [3]:
dataset.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


### Data Cleaning

#### Handling missing values
    
    - Handling Missing Values
    - Handling Duplicates
    - Check the Data type
    - Understanding the dataset

In [9]:
## Checking null values
dataset.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [10]:
## Remove unnecessary columns
dataset.drop('car_name',axis=1, inplace=True)
dataset.drop('brand',axis=1, inplace=True)

In [12]:
dataset.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [18]:
dataset.dtypes

model                 object
vehicle_age            int64
km_driven              int64
seller_type           object
fuel_type             object
transmission_type     object
mileage              float64
engine                 int64
max_power            float64
seats                  int64
selling_price          int64
dtype: object

In [21]:
## Getting All Different types of data in the dataset
numerical_features = [feature for feature in dataset.columns if dataset[feature].dtype != 'O']
categorical_features = [feature for feature in dataset.columns if dataset[feature].dtype == 'O']
discrete_features = [feature for feature in numerical_features if len(dataset[feature].unique()) <= 25]
continuous_features = [feature for feature in numerical_features if feature not in discrete_features]
print(f"Numerical Features: {len(numerical_features)}")
print(f"Categorical Features: {len(categorical_features)}")
print(f"Discrete Features: {len(discrete_features)}")
print(f"Continuous Features: {len(continuous_features)}")

Numerical Features: 7
Categorical Features: 4
Discrete Features: 2
Continuous Features: 5


In [22]:
## Independent and Dependent Features
from sklearn.model_selection import train_test_split
X = dataset.drop(['selling_price'],axis=1)
y = dataset['selling_price']

In [24]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


### Feature Encoding and Scaling

#### One Hot Encoding for Columns which had lesser unqiue values and not ordinal

 - One Hot Encoding is a process by which categorical variable are converted into a form that could be provided by ML algorithms to do a better job in prediction

In [25]:
len(dataset['model'].unique())

120

In [26]:
dataset['model'].value_counts()

model
i20             906
Swift Dzire     890
Swift           781
Alto            778
City            757
               ... 
Altroz            1
C                 1
Ghost             1
Quattroporte      1
Gurkha            1
Name: count, Length: 120, dtype: int64

In [27]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X['model'] = label_encoder.fit_transform(X['model'])

In [28]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [ ]:
## creating a column transformer

numerical_features = X.select_dtypes(exclude=['object']).columns
categorical_features = X.select_dtypes(include=['object']).columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_tranformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')


preprocessor = ColumnTransformer(
    [
        ("oneHotEncoder", categorical_transformer, categorical_features),
        ("standardScaler", numeric_tranformer, numerical_features)
    ], remainder='passthrough' ## to keep the columns which are not in the above transformers unchanged
)


In [30]:
X = preprocessor.fit_transform(X)

In [31]:
## Separate the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [32]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12328, 14), (3083, 14), (12328,), (3083,))